Simple code on how to read the .jsonl file from OpenAI batch output

In [2]:
import pandas as pd
import json

df_persona = pd.read_json(f"./persona_251113_51.jsonl", lines=True)

# --- 1. Extract persona text from nested dict in df_persona['response'] ---

def extract_persona_text(resp):
    # If response is a JSON string, try to parse it
    if isinstance(resp, str):
        try:
            resp = json.loads(resp)
        except Exception:
            return None

    try:
        return resp["body"]["output"][0]["content"][0]["text"]
    except (TypeError, KeyError, IndexError):
        return None

df_persona = df_persona.copy()
df_persona["text"] = df_persona["response"].apply(extract_persona_text)

# --- 2. Split custom_id into 'experiment' and 'participant' ---

df_persona[["experiment", "participant"]] = (
    df_persona["custom_id"].astype(str).str.split("|", n=1, expand=True)
)

The resulting dataframe `df_persona` basically consists of the persona text generated by GPT-5.1 for each participant in each experiment.

* `text` column refers to the persona of the player generated by GPT-5.1 (probably overfitted to the specific scenario it saw)
* `experiment` column refers to the id of the experiment (game)
* `participant` column refers to the id of the participant

In [3]:
df_persona.head()

,id,custom_id,response,error,text,experiment,participant
0,batch_req_69171f7ed070819093396edad7f77ef2,2CpmMrTnRYQhD69qN|DTZZoihgn3RAp6nBo,"{'status_code': 200, 'request_id': 'c62ac2e0a5...",NaN,You are an online game agent role-playing a mi...,2CpmMrTnRYQhD69qN,DTZZoihgn3RAp6nBo
1,batch_req_69171f7ec640819088568961977ace1d,2CpmMrTnRYQhD69qN|F5KawmecqZFsCNyFk,"{'status_code': 200, 'request_id': '5de1c8b0a9...",NaN,You are an online game-playing agent role-play...,2CpmMrTnRYQhD69qN,F5KawmecqZFsCNyFk
2,batch_req_69171f7ee4e48190b6f052f372d30a40,2CpmMrTnRYQhD69qN|bQirT5AvqT5wycWYj,"{'status_code': 200, 'request_id': '8b39291eff...",NaN,You are a human player participating in repeat...,2CpmMrTnRYQhD69qN,bQirT5AvqT5wycWYj
3,batch_req_69171f7eccbc81909b6420b61ad64a0a,2CpmMrTnRYQhD69qN|be9dLqkhDp6wZjbBo,"{'status_code': 200, 'request_id': '8db3852048...",NaN,You are an online game participant who takes s...,2CpmMrTnRYQhD69qN,be9dLqkhDp6wZjbBo
4,batch_req_69171f7ec91081908469e1521b79785f,2CpmMrTnRYQhD69qN|dgD9YndT5WFPZRgzi,"{'status_code': 200, 'request_id': 'd88fde7029...",NaN,You are an online game-playing agent participa...,2CpmMrTnRYQhD69qN,dgD9YndT5WFPZRgzi
